In [1]:
import astropy.time
from lsst.daf.butler import Butler, Timespan

In [33]:
from lsst.daf.butler import CollectionType

In [2]:
butler = Butler('/repo/embargo')
registry = butler.registry

In [3]:
data_path = '/home/j/jarugula/DM-37690/transfer_embargo/tests/data/'
test_from_butler = Butler(data_path+'test_from', writeable=True)
test_from_butler_registry = test_from_butler.registry

In [4]:
datasetType = 'calexp'
collections = 'LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616'
dataId = {'instrument':'LATISS'}

In [5]:
# testrefs = test_from_butler_registry.queryDatasets(datasetType, dataId=dataId, collections=collections)
# testrefs = test_from_butler_registry.queryDatasets(datasetType, dataId=dataId, collections=...)

# for i, ref in enumerate(testrefs):
#     print(ref.dataId.full)

In [6]:
now = astropy.time.Time.now().tai
embargo_hours = 80.0
embargo_period = astropy.time.TimeDelta(
        embargo_hours * 3600.0, format="sec"
    )
timespan_embargo = Timespan(now - embargo_period, None)

In [7]:
datarefs = registry.queryDimensionRecords(
            "exposure",
            dataId=dataId,
            datasets=datasetType,
            collections=collections,
            where="NOT exposure.timespan OVERLAPS timespan_embargo",
            bind={"timespan_embargo": timespan_embargo},
        )

In [8]:
for i, ref in enumerate(datarefs):
    print(ref.dataId)
    if i > 10:
        break

{instrument: 'LATISS', exposure: 2022110900640}
{instrument: 'LATISS', exposure: 2022110900620}
{instrument: 'LATISS', exposure: 2022110900761}
{instrument: 'LATISS', exposure: 2022110800658}
{instrument: 'LATISS', exposure: 2022110800699}
{instrument: 'LATISS', exposure: 2022110900454}
{instrument: 'LATISS', exposure: 2022110900438}
{instrument: 'LATISS', exposure: 2022111000436}
{instrument: 'LATISS', exposure: 2022110900673}
{instrument: 'LATISS', exposure: 2022110800580}
{instrument: 'LATISS', exposure: 2022110800312}
{instrument: 'LATISS', exposure: 2022110800341}


In [9]:
exposure_ids = [2022111000436, 2022110900673, 2022110800580]

In [10]:
datasetRefs = registry.queryDatasets(datasetType, dataId=dataId, collections=collections,
                                     where="exposure.id IN (exposure_ids)",
                                     bind={"exposure_ids": exposure_ids}).expanded()

In [11]:
for i,ref in enumerate(datasetRefs):
    print(ref.dataId)

{instrument: 'LATISS', detector: 0, visit: 2022110800580, ...}
{instrument: 'LATISS', detector: 0, visit: 2022110900673, ...}
{instrument: 'LATISS', detector: 0, visit: 2022111000436, ...}


In [18]:
for i,ref in enumerate(datasetRefs):
    print(ref)

calexp@{instrument: 'LATISS', detector: 0, visit: 2022110800580, ...} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z id=38721027-6eee-4dcf-a78c-f0a6b15f1611)
calexp@{instrument: 'LATISS', detector: 0, visit: 2022110900673, ...} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z id=18997382-c91c-4ec1-bb6b-24046f500de5)
calexp@{instrument: 'LATISS', detector: 0, visit: 2022111000436, ...} [sc=ExposureF] (run=LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z id=1b6977ab-fdab-4d5e-aa46-cda684ae9717)


In [ ]:
ref

In [12]:
test_from_butler.transfer_from(butler, source_refs=datasetRefs, transfer='copy',
                               skip_missing=True, register_dataset_types=True,
                               transfer_dimensions=True)

[DatasetRef(DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF), {instrument: 'LATISS', detector: 0, visit: 2022110800580, ...}, run='LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z', id=38721027-6eee-4dcf-a78c-f0a6b15f1611),
 DatasetRef(DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF), {instrument: 'LATISS', detector: 0, visit: 2022111000436, ...}, run='LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z', id=1b6977ab-fdab-4d5e-aa46-cda684ae9717),
 DatasetRef(DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF), {instrument: 'LATISS', detector: 0, visit: 2022110900673, ...}, run='LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z', id=18997382-c91c-4ec1-bb6b-24046f500de5)]

In [13]:
#get chained collection
collection_chain = registry.getCollectionChain(collections)

In [14]:
collection_chain

('LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z',
 'LATISS/raw/all',
 'LATISS/calib/DM-36719/bias.20221107',
 'LATISS/calib/DM-36719/dark.20221107',
 'LATISS/calib/DM-36719/flat-SDSSi.20221107',
 'LATISS/calib/DM-36719/flat-SDSSr.20221107',
 'LATISS/calib/DM-36719/flat-SDSSg.20221107',
 'LATISS/calib/DM-36484/bias.20221005a',
 'LATISS/calib/DM-36484/dark.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSg.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSr.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSi.20221006a',
 'u/czw/defects.20220608',
 'LATISS/calib/DM-33875',
 'u/czw/DM-28920/calib/bias.20210720',
 'u/czw/DM-28920/calib/dark.20210720a',
 'u/calib/DM-32209-20211013a-g',
 'u/calib/DM-32209-20211013a-felh',
 'u/czw/DM-28920/calib/flat.20210720',
 'u/czw/DM-28920/calib/defect.20210720a',
 'LATISS/calib/DM-28636',
 'LATISS/calib/DM-28636/unbounded',
 'refcats/DM-28636',
 'refcats/DM-33444',
 'skymaps')

In [53]:
for c in collection_chain:
    print(c)

LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z
LATISS/raw/all
LATISS/calib/DM-36719/bias.20221107
LATISS/calib/DM-36719/dark.20221107
LATISS/calib/DM-36719/flat-SDSSi.20221107
LATISS/calib/DM-36719/flat-SDSSr.20221107
LATISS/calib/DM-36719/flat-SDSSg.20221107
LATISS/calib/DM-36484/bias.20221005a
LATISS/calib/DM-36484/dark.20221006a
LATISS/calib/DM-36484/flat-SDSSg.20221006a
LATISS/calib/DM-36484/flat-SDSSr.20221006a
LATISS/calib/DM-36484/flat-SDSSi.20221006a
u/czw/defects.20220608
LATISS/calib/DM-33875
u/czw/DM-28920/calib/bias.20210720
u/czw/DM-28920/calib/dark.20210720a
u/calib/DM-32209-20211013a-g
u/calib/DM-32209-20211013a-felh
u/czw/DM-28920/calib/flat.20210720
u/czw/DM-28920/calib/defect.20210720a
LATISS/calib/DM-28636
LATISS/calib/DM-28636/unbounded
refcats/DM-28636
refcats/DM-33444
skymaps


In [21]:
registry.getCollectionSummary(collections)

CollectionSummary(dataset_types=NamedValueSet({DatasetType('transformSourceTable_metadata', {band, instrument, detector, physical_filter, visit}, TaskMetadata), DatasetType('consolidateSourceTable_metadata', {band, instrument, physical_filter, visit}, TaskMetadata), DatasetType('isr_log', {band, instrument, detector, physical_filter, exposure}, ButlerLogRecords), DatasetType('isr_metadata', {band, instrument, detector, physical_filter, exposure}, TaskMetadata), DatasetType('sourceTable_visit', {band, instrument, physical_filter, visit}, DataFrame), DatasetType('transformSourceTable_log', {band, instrument, detector, physical_filter, visit}, ButlerLogRecords), DatasetType('characterizeImage_log', {band, instrument, detector, physical_filter, visit}, ButlerLogRecords), DatasetType('calibrate_log', {band, instrument, detector, physical_filter, visit}, ButlerLogRecords), DatasetType('icExp', {band, instrument, detector, physical_filter, visit}, ExposureF), DatasetType('src', {band, instrum

In [24]:
test_from_butler_registry.getCollectionSummary('LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z')

CollectionSummary(dataset_types=NamedValueSet({DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF)}), governors={'instrument': {'LATISS'}})

In [54]:
# register all the collections from the chain
for c in collection_chain:
    test_from_butler_registry.registerCollection(c, CollectionType(3))

In [47]:
# test_from_butler_registry.removeCollection('test_chained_collection')

In [55]:
for c in sorted(test_from_butler_registry.queryCollections(datasetType='calexp')):
    print(c)

LATISS/calib/DM-28636
LATISS/calib/DM-28636/unbounded
LATISS/calib/DM-33875
LATISS/calib/DM-36484/bias.20221005a
LATISS/calib/DM-36484/dark.20221006a
LATISS/calib/DM-36484/flat-SDSSg.20221006a
LATISS/calib/DM-36484/flat-SDSSi.20221006a
LATISS/calib/DM-36484/flat-SDSSr.20221006a
LATISS/calib/DM-36719/bias.20221107
LATISS/calib/DM-36719/dark.20221107
LATISS/calib/DM-36719/flat-SDSSg.20221107
LATISS/calib/DM-36719/flat-SDSSi.20221107
LATISS/calib/DM-36719/flat-SDSSr.20221107
LATISS/raw/all
LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616
LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z
refcats/DM-28636
refcats/DM-33444
skymaps
u/calib/DM-32209-20211013a-felh
u/calib/DM-32209-20211013a-g
u/czw/DM-28920/calib/bias.20210720
u/czw/DM-28920/calib/dark.20210720a
u/czw/DM-28920/calib/defect.20210720a
u/czw/DM-28920/calib/flat.20210720
u/czw/defects.20220608


In [56]:
test_from_butler_registry.getCollectionChain(collections)

()

In [57]:
# add chained collection to the destination butler
test_from_butler_registry.setCollectionChain(parent=collections, children=collection_chain)

In [66]:
test_from_butler_registry.getCollectionChain(collections)

('LATISS/runs/AUXTEL_DRP_IMAGING_2022-11A/w_2022_46/PREOPS-1616/20221111T182132Z',
 'LATISS/raw/all',
 'LATISS/calib/DM-36719/bias.20221107',
 'LATISS/calib/DM-36719/dark.20221107',
 'LATISS/calib/DM-36719/flat-SDSSi.20221107',
 'LATISS/calib/DM-36719/flat-SDSSr.20221107',
 'LATISS/calib/DM-36719/flat-SDSSg.20221107',
 'LATISS/calib/DM-36484/bias.20221005a',
 'LATISS/calib/DM-36484/dark.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSg.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSr.20221006a',
 'LATISS/calib/DM-36484/flat-SDSSi.20221006a',
 'u/czw/defects.20220608',
 'LATISS/calib/DM-33875',
 'u/czw/DM-28920/calib/bias.20210720',
 'u/czw/DM-28920/calib/dark.20210720a',
 'u/calib/DM-32209-20211013a-g',
 'u/calib/DM-32209-20211013a-felh',
 'u/czw/DM-28920/calib/flat.20210720',
 'u/czw/DM-28920/calib/defect.20210720a',
 'LATISS/calib/DM-28636',
 'LATISS/calib/DM-28636/unbounded',
 'refcats/DM-28636',
 'refcats/DM-33444',
 'skymaps')

In [67]:
test_from_butler_registry.getCollectionSummary(collections)

CollectionSummary(dataset_types=NamedValueSet({DatasetType('calexp', {band, instrument, detector, physical_filter, visit}, ExposureF), DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure)}), governors={'instrument': {'LATISS'}})

In [58]:
testrefs = test_from_butler_registry.queryDatasets(datasetType, dataId=dataId, collections=collections)
for i, ref in enumerate(testrefs):
    print(ref.dataId.full)

{band: 'i', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSi_65mm~empty', visit: 2022110800580}
{band: 'g', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSg_65mm~empty', visit: 2022110900673}
{band: 'i', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSi_65mm~empty', visit: 2022111000436}


In [59]:
testrefs = test_from_butler_registry.queryDatasets(datasetType, dataId=dataId, collections=...)
for i, ref in enumerate(testrefs):
    print(ref.dataId.full)

{band: 'i', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSi_65mm~empty', visit: 2022110800580}
{band: 'g', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSg_65mm~empty', visit: 2022110900673}
{band: 'i', instrument: 'LATISS', detector: 0, physical_filter: 'SDSSi_65mm~empty', visit: 2022111000436}


In [64]:
# testrefs = test_from_butler_registry.queryDimensionRecords("exposure",  dataId=dataId, datasets=datasetType,collections=...)
testrefs = test_from_butler_registry.queryDimensionRecords("visit",  dataId=dataId, datasets=datasetType,collections=collections)
for i, ref in enumerate(testrefs):
    print(ref)

visit:
  instrument: 'LATISS'
  id: 2022110800580
  physical_filter: 'SDSSi_65mm~empty'
  name: 'AT_O_20221108_000580'
  day_obs: 20221108
  seq_num: 580
  exposure_time: 30.0
  target_name: 'SPT-CL-J0559-5249_063'
  observation_reason: 'object'
  science_program: 'AUXTEL_DRP_IMAGING'
  azimuth: 139.27777238604904
  zenith_angle: 36.25017562667199
  region: ConvexPolygon([UnitVector3d(-0.0014031665810288064, 0.6073223384722833, -0.7944542833392637), UnitVector3d(-0.0035242917214096405, 0.6073137483105336, -0.7944542721144321), UnitVector3d(-0.003530996614655318, 0.6089710242215567, -0.7931846088531064), UnitVector3d(-0.0014098714605740878, 0.608979614297738, -0.7931846201435344)])
  timespan: Timespan(begin=astropy.time.Time('2022-11-09 04:54:12.359010', scale='tai', format='iso'), end=astropy.time.Time('2022-11-09 04:54:42.592000', scale='tai', format='iso'))
visit:
  instrument: 'LATISS'
  id: 2022110900673
  physical_filter: 'SDSSg_65mm~empty'
  name: 'AT_O_20221109_000673'
  day_ob

In [65]:
testrefs = test_from_butler_registry.queryDimensionRecords("exposure",  dataId=dataId, datasets=datasetType,collections=collections)
for i, ref in enumerate(testrefs):
    print(ref)

In [18]:
outside_embargo = [
            dt.id
            for dt in test_from_butler_registry.queryDimensionRecords(
                "exposure",
                dataId=dataId,
                datasets=datasetType,
                # collections=collections_exposure,
                collections=...,)]

In [19]:
outside_embargo

[]

In [20]:
outside_embargo = [
            dt.id
            for dt in test_from_butler_registry.queryDimensionRecords(
                "exposure",
                dataId=dataId,
                datasets=datasetType,
                # collections=collections_exposure,
                collections=...,
                where="NOT exposure.timespan OVERLAPS\
                                                        timespan_embargo",
                bind={"timespan_embargo": timespan_embargo},
            )
        ]

In [21]:
outside_embargo

[]